# Export MNIST

read images and labels in ubyte format

In [35]:
import gzip
import numpy as np
import struct

max_size = 18

# images_path = '../../datasets/mnist/t10k-images-idx3-ubyte.gz'
# labels_path = '../../datasets/mnist/t10k-labels-idx1-ubyte.gz'
# out_path = '../../datasets/mnist/test/'

# images_path = '../../datasets/mnist/train-images-idx3-ubyte.gz'
# labels_path = '../../datasets/mnist/train-labels-idx1-ubyte.gz'
# out_path = '../../datasets/mnist/train/'

# https://stackoverflow.com/questions/39969045/parsing-yann-lecuns-mnist-idx-file-format

with gzip.open(images_path,'rb') as f:
    magic, size = struct.unpack(">II", f.read(8))
    nrows, ncols = struct.unpack(">II", f.read(8))
    images = np.frombuffer(f.read(), dtype=np.dtype(np.uint8).newbyteorder('>'))
    images = images.reshape((size, nrows, ncols))


with gzip.open(labels_path,'rb') as f:
    magic, size = struct.unpack(">II", f.read(8))
    labels = np.frombuffer(f.read(), dtype=np.dtype(np.uint8).newbyteorder('>'))

define helper function to generate polygon for the images

In [36]:
import cv2


def image2poly(image, img_size=28, max_size=10):
    cnt, _ = cv2.findContours(
        np.array(image), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    cnt = max(cnt, key=cv2.contourArea)
    poly = cv2.approxPolyDP(
        cnt, 0.015 * cv2.arcLength(cnt, True), True) / img_size
    poly = poly.reshape(-1, 2)[:max_size]

    poly = np.c_[np.arange(1, len(poly) + 1) / (len(poly) + 1), poly]
    y = np.array([[0, 0, 0]])
    z = np.array([[1, poly[0, 1], poly[0, 2]]])
    poly = np.r_[y, poly, z]

    length = len(poly)
    real_size = max_size + 2

    for i in range(0, real_size - length):
        poly = np.r_[poly, z]

    if (len(poly) != real_size):
        print("Diff")
    return poly, length

export images to png

In [37]:
from PIL import Image
from os.path import join

dataset_entries = []

idx = 0
for image, label in zip(images, labels):
    im = Image.fromarray(image)
    file_path = join("images", f"img{idx:05}_{label}.png")
    poly, length = image2poly(image, max_size=max_size)
    poly = poly.flatten().tolist()
    poly = ','.join(str(n) for n in poly)

    dataset_entries.append({
        "file_path": file_path,
        "label": label,
        "length": length,
        "polygon": poly
    })

    # im.save(join(out_path, file_path))
    idx += 1

save label and polygons to csv with an association to the file path they refer to

In [38]:
import csv

with open(join(out_path, "polygon-mnist2.csv"), 'w', newline='') as file: 
    writer = csv.DictWriter(file, fieldnames = ["file_path", "label", "length", "polygon"])
    writer.writeheader() 
    writer.writerows(dataset_entries)